# Pre-process input data for coastal variable extraction

Author: Emily Sturdivant; esturdivant@usgs.gov

***

You must run this notebook within the ArcGIS Pro conda environment. To do so, type the following in your command prompt (assuming it has the default set-up and substituting path\to\dir with the location of the repository):

```bat
cd path\to\dir\BI-geomorph-extraction
\ArcGIS\Pro\bin\Python\Scripts\proenv
jupyter notebook
```

## Pre-processing steps

1. Pre-created geomorphic features: dunes, shoreline points, armoring.
2. Inlets
3. Shoreline
4. Transects - extend and sort
5. Transects - tidy

#### Import modules

Sometimes the user needs to run the code through the ArcPy console (when it involves selecting layers). --- an alternative may be to use the Pro version of the mapping toolbox and say current map document... --- In this case, we need to make sure that ArcGIS can find this package. 

In [1]:
import os
import sys
import time
import shutil
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import arcpy
if os.path.basename(os.getcwd()) == 'CoastalVarExtractor':
    print('Module path is: {}'.format(os.getcwd()))
    import CoastalVarExtractor.functions_warcpy as fwa

Module path is: C:\Users\esturdivant\Code\BI-geomorph-extraction\CoastalVarExtractor


In [2]:
mod_path = r"C:\Users\esturdivant\Code\BI-geomorph-extraction" # replace with path to module
sys.path.append(mod_path)
import CoastalVarExtractor.functions_warcpy as fwa

## Initialize variables

Based on the project directory, and the site and year you have input, setvars.py will set a bunch of variables as the names of folders, files, and fields. 1) set-up the project folder and paths: 

In [3]:
from CoastalVarExtractor.setvars import *
from CoastalVarExtractor.configmap import *
proj_dir = r"\\Mac\stor\Projects\TransectExtraction\Fisherman2014"
site = "Fisherman"
year = "2014"
SiteYear_strings = siteyear[site+year] # get siteyear dict from configmap
home = os.path.join(proj_dir, '{site}{year}.gdb'.format(**SiteYear_strings))
arcpy.env.workspace = home
arcpy.env.scratchWorkspace = proj_dir

setvars.py initialized variables.


In [4]:
extendedTrans = 'extTrans'
inletLines = 'inletLines'
ShorelinePts = 'SLpts'
dlPts = 'DLpts'
dhPts = 'DHpts'
armorLines = 'armorLines'
elevGrid = 'DEM'
elevGrid_5m = 'DEM_5m'

extendedTrans = os.path.join(home, 'extTrans')
inletLines = os.path.join(home, 'inletLines')
ShorelinePts = os.path.join(home, 'SLpts')
dlPts = os.path.join(home, 'DLpts')
dhPts = os.path.join(home, 'DHpts')
armorLines = os.path.join(home, 'armorLines')
elevGrid = os.path.join(home, 'DEM')
elevGrid_5m = os.path.join(home, 'DEM_5m')

## Dunes and armoring <a name="geofeatures"></a>
Display the points and the DEM in a GIS to check for irregularities. For example, if shoreline points representing a distance less than X m are visually offset from the general shoreline, they should likely be removed. Another red flag is when the positions of dlows and dhighs in relation to the shore are illogical, i.e. dune crests are seaward of dune toes. 

#### Replace fill values with Null. 

In [ ]:
fwa.ReplaceValueInFC(dhPts, oldvalue=fill, newvalue=None, fields=["dhigh_z"])      # Dhighs
fwa.ReplaceValueInFC(dlPts, oldvalue=fill, newvalue=None, fields=["dlow_z"])       # Dlows
fwa.ReplaceValueInFC(ShorelinePts, oldvalue=fill, newvalue=None, fields=["slope"]) # Shoreline

#### Armoring
If the dlows do not capture the entire top-of-beach due to atypical formations caused by anthropogenic modification, you may need to digitize the beachfront armoring. The next code block will generate an empty feature class. Refer to the DEM and orthoimagery. If there is no armoring in the study area, continue. If there is armoring, use the Editing toolbar to add lines to the feature class that trace instances of armoring. Common manifestations of what we call armoring are sandfencing and sandbagging and concrete seawalls. 

*__Requires manipulation in GIS__*

In [ ]:
arcpy.CreateFeatureclass_management(home, armorLines, 'POLYLINE', spatial_reference=utmSR)
print("{} created. Now manually digitize the shorefront armoring.".format(armorLines))

## Inlets
We also need to manually digitize inlets if an inlet delineation does not already exist. To do, the code will produce the feature class. After which use the Editing toolbar to create a line when the island shore meets a tidal inlet. If the study area includes both sides of an inlet, that inlet will be represented by two lines. The inlet lines are use to define the bounds of the oceanside shore, which is also considered the point where the oceanside shore meets the bayside. Inlet lines must intersect the MHW contour. 

*__Requires manipulation in GIS__*

In [ ]:
# manually create lines that correspond to end of land and cross the MHW line (use bndpoly/DEM)
arcpy.CreateFeatureclass_management(home, inletLines, 'POLYLINE', spatial_reference=utmSR)
print("{} created. Now we'll stop for you to manually create lines at each inlet.".format(inletLines))

## Shoreline
The shoreline is produced through a combination of the DEM and the shoreline points. The first step converts the DEM to both MTL and MHW contour polygons. Those polygons are combined to produce the full shoreline, which is considered to fall at MHW on the oceanside and MTL on the bayside (to include partially submerged wetland). User input is required to identify only the areas within the study area and eliminate isolated landmasses that are not. 

*__Requires display in GIS__*

In [ ]:
bndpoly = fwa.DEMtoFullShorelinePoly(elevGrid_5m, MTL, MHW, inletLines, ShorelinePts)
print('Select features from {} that should not be included in {}'.format(bndpoly, barrierBoundary))

In [ ]:
bndMTL = 'bndpoly_mtl'
bndMHW = 'bndpoly_mhw'
bndpoly = 'bndpoly'
print("Creating the MTL contour polgon from the DEM...")
RasterToLandPerimeter(elevGrid, bndMTL, MTL)  # Polygon of MTL contour
print("Creating the MHW contour polgon from the DEM...")
RasterToLandPerimeter(elevGrid, bndMHW, MHW)  # Polygon of MHW contour

In [ ]:
print("Combining the two polygons...")
bndpoly = fwa.CombineShorelinePolygons(bndMTL, bndMHW, inletLines, ShorelinePts, bndpoly)

Once the features to delete are selected, either delete in the GIS or run the code.
__Do not:__ Select the features in ArcGIS and then run DeleteFeatures from here. That will delete the entire feature class. 

In [ ]:
# Change bndpoly to match the layer name before running.
bndpoly = bndpoly
arcpy.DeleteFeatures_management(bndpoly)

The next step snaps the boundary polygon to the shoreline points anywhere they don't already match and as long as as they are within 25 m of each other. 

In [ ]:
bndpoly = 'bndpoly'
barrierBoundary = fwa.NewBNDpoly(bndpoly, ShorelinePts, barrierBoundary, '25 METERS', '50 METERS')
print("Created: '{}'".format(barrierBoundary))

In [5]:
shoreline = fwa.CreateShoreBetweenInlets(barrierBoundary, inletLines, shoreline, 
                                         ShorelinePts, proj_code)

Preserving only those line segments that intersect shoreline points...
Dissolving the line to create Fisherman2014_ShoreBetweenInlets...


After this step, you'll want to make sure the shoreline looks okay. There should be only one line segment for each stretch of shore between two inlets. Segments may be incorrectly deleted if the shoreline points are missing in the area. Segments may be incorrectly preserved if they are intersect a shoreline point. To rectify, either perform manual editing or rerun this code with modifications. 

## Transects
### Extended transects

Create extendedTrans, LT transects with gaps filled and lines extended

#### 1. Extend and Copy only the geometry of transects to use as material for filling gaps

In [ ]:
fwa.ExtendLine(fc=orig_trans, new_fc=trans_extended, distance=extendlength, proj_code=proj_code)
fwa.CopyAndWipeFC(trans_extended, trans_presort, ['sort_ID'])
print("MANUALLY: use groups of existing transects in new FC '{}' to fill gaps.".format(trans_presort))

*__Requires manipulation in GIS__*

Manually use groups of existing transects in the new FC to fill gaps. Avoid overlapping transects as much as possible. The new FC is only for use as spatial filler. As a result, the attribute values don't matter and there is no need to preserve the original transects (i.e. you can simply move transects instead of duplicating them). However, do not slightly move any transects. If they are moved, they will not be deleted in the next stage. If you slightly move any, you can either undo or delete that line entirely.

*__PrepTransects_part2 uses SelectLayerByLocation so it must be executed in the ArcPy console.__*

In [ ]:
fwa.PrepTransects_part2(trans_presort, trans_extended, barrierBoundary)

#### 2. Sort the transects.

First set the sorting parameters. If the shoreline curves, the GIS will not correctly establish the alongshore order by simple ordering from the identified sort_corner so you need to identify different groups of transects for sorting. If this is the case, answer yes to the next prompt.

In [ ]:
multi_sort = input("Do we need to sort the transects in batches to preserve the order? (y/n) ") 
if multi_sort == 'y':
    arcpy.CreateFeatureclass_management(trans_dir, sort_lines, "POLYLINE", spatial_reference=utmSR)
    print("MANUALLY: Add features to sort_lines. Add the features in the \
    order they should be sorted and indicate the sort corner in the field sort_corn.")
else:
    sort_lines = []
    # Corner from which to start sorting, LL = lower left, etc. 
    sort_corner = input("Sort corner (LL, LR, UL, UR): ")  

*__Requires manipulation in GIS__*

If the transects need to be sorted in batches to preserve the order, then we generated an empty sort lines feature class. Add lines to the FC that will be used to iteratively select the intersecting transects and order them based on the sort corner you will indicate next. Add fields sort and sort_corn to the FC and assign values to order the transects. sort_corn indicates the sort corner for the group pertaining to each line. 

*__SortTransectsFromSortLines uses SelectLayerByLocation so it must be executed in the ArcPy console.__*

In [ ]:
fwa.SortTransectsFromSortLines(trans_presort, extendedTrans, sort_lines, 
                               sortfield=tID_fld, sort_corner=sort_corner)
# # Clean up OBJECTID
# if len(arcpy.ListFields(extendedTrans, 'OBJECTID*')) == 2:
#     fwa.ReplaceFields(extendedTrans, {'OBJECTID': 'OID@'})

### Tidy the extended transects

If extended transects overlap, they need to be semi-manually adjusted ('tidied') so that there is no overlap. Overlapping transects cause problems when splitting the transects into 5-m points and when converting the transects to rasters. The user needs to select the boundary lines between groups of overlapping transects. 

*__Requires manipulation in GIS__*

The selected transects are copied into a separate FC and used to split the other transects at the intersection points. Once the lines are split, the user manually deletes the undesired segments. These will be the ones that are to the bayside of the boundary lines.

In [ ]:
# Copy only the selected lines
arcpy.CopyFeatures_management(orig_extTrans, overlapTrans_lines)          # copy selected transects
arcpy.SelectLayerByAttribute_management(orig_extTrans, "CLEAR_SELECTION") # clear the selection

# Split transects at the lines of overlap.
arcpy.Intersect_analysis([orig_extTrans, overlapTrans_lines], trans_x,
                         'ALL', output_type="POINT")
arcpy.SplitLineAtPoint_management(orig_extTrans, trans_x, extTrans_tidy)
print("MANUALLY: Select transect segments to be deleted. ")

Resume after manual selection or delete manually.

In [ ]:
arcpy.DeleteFeatures_management(extTrans_tidy)
print("Pre-processing completed.")